In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
!pip install efficientnet_pytorch

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
# from tqdm.auto import tqdm
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import torchvision.models as models
from torch.autograd import Variable
import torch.optim as optim
import torchvision
# from torchsummary import summary
import gc
from efficientnet_pytorch import EfficientNet


dataset_dir = '../dataset'
#dataset_dir = '/kaggle/input/bengaliai-cv19'
train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')

# 前処理を関数にまとめた
def resize(X, out_height=64, out_width=64):
    print('Resizing raw image... / 前処理実行中…')
    # resized = {} # 前処理された画像が格納されるリスト
    resized = np.zeros((len(X), out_height * out_width))

    for i in range(len(X)):
        image = X.iloc[[i]].values.reshape(HEIGHT, WIDTH)
        _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        left = 1000
        right = -1
        top = 1000
        bottom = -1

        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            left = min(x, left)
            right = max(x+w, right)
            top = min(y, top)
            bottom = max(y+h, bottom)

        roi = image[top:bottom, left:right]
        resized_roi = cv2.resize(roi, (out_height, out_width),interpolation=cv2.INTER_AREA)
        #resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_GRAY2RGB) # efficientのinputになるため３次元化
        #resized_roi = np.transpose(np.array(resized_roi), (2,0,1)) -> (3,224,224)への変換をここでやるとメモリクラッシュするのでmydataset()の__get_item__()内でやる
        resized[i] = resized_roi.reshape(-1)
    return resized

# PyTorch式のデータセットクラスを定義

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, Y, transform=None):
        self.transform = transform
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        out_data = cv2.resize(self.X[idx].reshape(64,64), (224, 224),interpolation=cv2.INTER_AREA)
        out_data = out_data.reshape(224, 224, 1)
        out_data = cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB)
        out_data = np.transpose(out_data, (2,0,1)) / 255 # shapeのcv2変換が終わってから /255 をやらないとエラー出る
        out_data = out_data.reshape(3,224,224) 
        out_data = torch.tensor(out_data, dtype=torch.float)

        #root_label = torch.tensor(self.Y[0][idx], dtype=torch.long)
        #vowel_label = torch.tensor(self.Y[1][idx], dtype=torch.long)
        #cons_label = torch.tensor(self.Y[2][idx], dtype=torch.long)

        root_label = torch.tensor(np.argmax(self.Y[0][idx]), dtype=torch.long)
        vowel_label = torch.tensor(np.argmax(self.Y[1][idx]), dtype=torch.long)
        cons_label = torch.tensor(np.argmax(self.Y[2][idx]), dtype=torch.long)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, root_label, vowel_label, cons_label

def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

class model(nn.Module):
    def __init__(self):
        #resnet18の実装
        super(model, self).__init__()
        self.efficient_imagenet = EfficientNet.from_pretrained('efficientnet-b0')
        self.fc = nn.Linear(1000, 512)

        self.head_root = nn.Linear(512, 168) # + softmax
        self.head_vowel = nn.Linear(512, 11) # + softmax
        self.head_consonant = nn.Linear(512, 7) # + softmax
    
    def forward(self, x):
        x = self.efficient_imagenet(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        head_root = self.head_root(x)
        head_vowel = self.head_vowel(x)
        head_consonant = self.head_consonant(x)

        return head_root, head_vowel, head_consonant # not sure..
model = model()
#model = EfficientNet.from_pretrained('efficientnet-b0')
model = try_gpu(model)

criterion1 = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

def train(model, epoch, train_loader):
    model.train()
    print(f'Epoch number {epoch}')
    correct_r, correct_v, correct_c = 0, 0, 0
    total_r, total_v, total_c = 0, 0, 0

    for i, data in enumerate(train_loader, 0):
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()

        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        total_r += root_l.size(0)
        correct_r += (root_pred == root_l).sum()
        total_v += vowel_l.size(0)
        correct_v += (vowel_pred == vowel_l).sum()
        total_c += consonant_l.size(0)
        correct_c += (consonant_pred == consonant_l).sum()
        # if i % 500 == 0:
        #     print("epoch{} root {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss1.data))
        #     print("epoch{} vowel {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss2.data))
        #     print("epoch{} consonant {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss3.data))
    
    print("root Accuracy {}/{} {:.2f}%".format(correct_r, total_r, 100.0*correct_r/total_r))
    print("vowel Accuracy {}/{} {:.2f}%".format(correct_v, total_v, 100.0*correct_v/total_v))
    print("consonant Accuracy {}/{} {:.2f}%".format(correct_c, total_c, 100.0*correct_c/total_c))


def test(model, test_loader):
    model.eval()
    correct_r, correct_v, correct_c = 0, 0, 0
    total_r, total_v, total_c = 0, 0, 0
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        total_r += root_l.size(0)
        correct_r += (root_pred == root_l).sum()
        total_v += vowel_l.size(0)
        correct_v += (vowel_pred == vowel_l).sum()
        total_c += consonant_l.size(0)
        correct_c += (consonant_pred == consonant_l).sum()

# 訓練ループ / Training Loop

val_perc = 0.2  # validation set の割合（クロスバリデーション）
epochs = 1 #4

#training loop
for parq_i in range(4):
    print('=============================')
    print(f'Parquet {parq_i} の訓練を開始')

    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)

    HEIGHT = 137
    WIDTH = 236

    X_train = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])

    #X_train_resized = resize(X_train).T / 255.0   # 値を0~1におさめる
    X_train_resized = resize(X_train).astype(np.uint8) # astype(np.uint8)をしてあげることで後で cv2.cvtColor(out_data, cv2.COLOR_GRAY2RGB) が実行できるようになる
    print("X_train_resized.shape", X_train_resized.shape)

    # メモリ節約
    del X_train

    # PyTorchのデータセットクラスを作る前に、ラベルの情報も整備
    # 1-of-K符号化とか、One Hot Encodingとか呼ばれる方法でラベルをつくる

    # 注：　PyTorchでは、ラベルはOne Hotじゃなくて良いことが判明したので、結局 MyDataset で元のラベルに戻している

    Y_train_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_train_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    Y_train = [Y_train_root, Y_train_vowel, Y_train_cons]

    trainval_dataset = MyDataset(X_train_resized, Y_train)

    n_samples = len(trainval_dataset)

    train_size = int(len(trainval_dataset)*(1.0 - val_perc))
    val_size = n_samples - train_size
    print(f'train size: {train_size}, validation size: {val_size}')

    train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

    train_loader = DataLoader(dataset=trainval_dataset,
                        batch_size=32, shuffle=True, num_workers=4)
    val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=0)

    for i in range(1,epochs+1):
        train(model, i, train_loader)
        test(model, val_loader)

    # メモリ節約
    del train_df_with_img
    del train_dataset
    del trainval_dataset
    del train_loader
    del X_train_resized
    gc.collect()

# 提出ファイルの準備

target=[]
row_id=[] # row_id place holder

for parq_i in range(4):
    df_test_img = pd.read_parquet(dataset_dir + f'/test_image_data_{parq_i}.parquet')
    # df_test_img = pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet') # Error Check!
    df_test_img.set_index('image_id', inplace=True)

    X_test_resized = resize(df_test_img).astype(np.uint8)
    X_test_resized = np.array([cv2.resize(X_test.reshape(64,64), (224, 224),interpolation=cv2.INTER_AREA) for X_test in X_test_resized])
    X_test_resized = np.array([X_test.reshape(224,224,1) for X_test in X_test_resized])
    X_test_resized = np.array([cv2.cvtColor(X_test, cv2.COLOR_GRAY2RGB) for X_test in X_test_resized])
    X_test_resized = np.array([np.transpose(X_test, (2,0,1)) for X_test in X_test_resized])
    X_test_resized = X_test_resized / 255
    X_test_resized = X_test_resized.reshape(-1, 3, 224, 224)

    test_inputs = torch.tensor(X_test_resized, dtype=torch.float)
    test_inputs = Variable(test_inputs)
    test_inputs = try_gpu(test_inputs)

    del X_test_resized
    gc.collect()
    
# テストデータをいっぺんに入れるとメモリが足りないので変更！
#     root_o, vowel_o, consonant_o = model(test_inputs)
#     root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
#     if torch.cuda.is_available():
#         root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))
#
#     root_pred, vowel_pred, consonant_pred = root_pred.numpy(), vowel_pred.numpy(), consonant_pred.numpy()
#
#     for k, id in enumerate(df_test_img.index.values):
#         row_id.append(id+'_consonant_diacritic')
#         target.append(consonant_pred[k])
#         row_id.append(id+'_grapheme_root')
#         target.append(root_pred[k])
#         row_id.append(id+'_vowel_diacritic')
#         target.append(vowel_pred[k])

    for k, id in enumerate(df_test_img.index.values):
        data = test_inputs[k].reshape(1, 3, 224, 224)
        root_o, vowel_o, consonant_o = model(data)
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        if torch.cuda.is_available():
            root_pred, vowel_pred, consonant_pred = root_pred.to(torch.device("cpu")), vowel_pred.to(torch.device("cpu")), consonant_pred.to(torch.device("cpu"))

        root_pred, vowel_pred, consonant_pred = root_pred.item(), vowel_pred.item(), consonant_pred.item()
        
        row_id.append(id+'_consonant_diacritic')
        target.append(consonant_pred)
        row_id.append(id+'_grapheme_root')
        target.append(root_pred)
        row_id.append(id+'_vowel_diacritic')
        target.append(vowel_pred)
    
    del df_test_img
    gc.collect()

df_sample = pd.DataFrame(
    {
        'row_id': row_id,
        'target':target
    },
    columns = ['row_id','target'] 
)

df_sample.to_csv('submission.csv',index=False)
df_sample.head(36)
# 23:25 実行　→　24:50

Loaded pretrained weights for efficientnet-b0
Parquet 0 の訓練を開始
Resizing raw image... / 前処理実行中…
X_train_resized.shape (50210, 4096)
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 33066/50210 65.86%
vowel Accuracy 44378/50210 88.38%
consonant Accuracy 45380/50210 90.38%
Parquet 1 の訓練を開始
Resizing raw image... / 前処理実行中…
X_train_resized.shape (50210, 4096)
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 41089/50210 81.83%
vowel Accuracy 47193/50210 93.99%
consonant Accuracy 47464/50210 94.53%
Parquet 2 の訓練を開始
Resizing raw image... / 前処理実行中…
X_train_resized.shape (50210, 4096)
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 42442/50210 84.53%
vowel Accuracy 47648/50210 94.90%
consonant Accuracy 47807/50210 95.21%
Parquet 3 の訓練を開始
Resizing raw image... / 前処理実行中…
X_train_resized.shape (50210, 4096)
train size: 40168, validation size: 10042
Epoch number 1
root Accuracy 43199/50210 86.04%
vowel Accuracy 47926/50210 95.45%
cons

,row_id,target
0,Test_0_consonant_diacritic,0
1,Test_0_grapheme_root,3
2,Test_0_vowel_diacritic,0
3,Test_1_consonant_diacritic,0
4,Test_1_grapheme_root,93
5,Test_1_vowel_diacritic,2
6,Test_2_consonant_diacritic,0
7,Test_2_grapheme_root,19
8,Test_2_vowel_diacritic,0
9,Test_3_consonant_diacritic,0


In [5]:
!ps aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39192  1512 ?        Ss   12:28   0:00 /bin/bash -e 
root           9  0.0  0.1 685204 38984 ?        Sl   12:28   0:03 /tools/node/b
root          19  0.1  0.3 479204 81284 ?        Sl   12:28   0:05 /usr/bin/pyth
root         134  0.0  0.0  35888  1312 ?        Ss   12:28   0:00 tail -n +0 -F
root         183  0.0  0.0  18376   304 ?        S    12:31   0:00 /bin/bash --n
root         184  0.0  0.0 2173148 2268 ?        Sl   12:31   0:00 /opt/google/d
root         185  0.0  0.0  11596   280 ?        S    12:31   0:00 grep --color=
root         282  0.8  0.2 2421620 61992 ?       Sl   12:32   0:24 /opt/google/d
root         293  0.0  0.0      0     0 ?        Z    12:32   0:00 [fusermount] 
root         327  0.0  0.0  18376   256 ?        S    12:32   0:00 bash -c tail 
root         328  0.0  0.0   4568    64 ?        S    12:32   0:00 tail -n +0 -F
root         329  0.0  0.0  11464 